In [1]:
# Import necessary libraries
import numpy as np, pandas as pd
import random
import time
import os

# Adjust options for viewing dataframes in notebook
pd.set_option('display.width', 2000)

In [2]:
# Function for reading a data file

def read_data(name):
    df = pd.read_csv('./../../data/clean_data/' + name + '.csv')
    df['Hour'] = pd.to_datetime(df['Hour'])
    df = df.set_index('Hour')
    df.index.freq = 'h'
    return df

In [3]:
# Function for locating random batches from dataset

## Take dataset, length of test data and number of batches to extract as parameters
## Find a number of digits that represent the indices of the original dataset, where the extracted batches will start from
## Each subset dataset will not intersect with other datasets

def extract_batches(df, test_length, batch_count):
    start = time.time()
    random_int_list = []
    m = 0
    random.seed(1)
    data_end = len(df)
    while m < batch_count:
        random_int = random.randint(0, data_end - test_length)
        q = 0
        for i in random_int_list:
            # Make sure batches do not overlap
            if i + test_length <= random_int or random_int + test_length <= i:
                pass
            else:
                q = 1
                # In case solution does not exist with current combination
                if time.time() - start > 2:
                    print('Restarted')
                    random_int_list = []
                    start = time.time()
                    m = 0

        # Prevent batches with 0 energy
        if df['Energy'][random_int: random_int + test_length].sum() == 0:
            q = 1

        # Record the position of the batch
        if q == 0:
            random_int_list.append(random_int)
            m = m + 1
    return random_int_list

In [4]:
# Range Pricing Method:
# The energy rate assigned to an hour's demand value is dependent on which demand threshold it falls under
# Demand thresholds are calculated by splitting the demand range evenly
# The higher the demand threshold the higher the price assigned

# Function for creating a price profile using the range pricing method

def range_pricing(test, min_price, max_price):

    # Possible prices per kWh are in range [min_price, max_price]
    price_set = [x / 100 for x in range(min_price, max_price + 1)]
    price_set_count = len(price_set)

    # The range of demand is split into equal bins, one for each possible price
    max_demand = max(test)
    min_demand = min(test)
    bin_size = (max_demand - min_demand) / price_set_count

    # Demand thresholds for each price are calculated
    demand_thresholds = []
    j = 1
    while j < price_set_count + 1:
        demand_thresholds.append(min_demand + bin_size * j)
        j = j + 1

    # To tackle float representation imprecision, the highest demand threshold is increased a little
    demand_thresholds[-1] = demand_thresholds[-1] + 1

    # Every demand value is compared with the demand thresholds and assigned a demand-proportional price
    price_profile = []
    i = 0
    while i < len(test):
        j = 0
        while j < price_set_count:
            if test[i] <= demand_thresholds[j]:
                price_profile.append(price_set[j])
                break
            j = j + 1
        i = i + 1

    return price_profile

In [5]:
# Percentile Pricing Method:
# The energy rate assigned to an hour's demand value is dependent on which demand threshold it falls under
# Demand thresholds are calculated by identifying demand values at specific percentile positions
# The higher the demand threshold the higher the price assigned

# Function for creating a price profile using the percentile pricing method

def percentile_pricing(test, min_price, max_price):

    # A list of possible prices in range [min_price, max_price] is created
    price_set = [x / 100 for x in range(min_price, max_price + 1)]
    price_set_count = len(price_set)

    # Percentile thresholds for each price are calculated
    bin_size = 100 / price_set_count
    percentile_thresholds = []
    j = 1
    while j < price_set_count + 1:
        percentile_thresholds.append(bin_size * j)
        j = j + 1

    # Demand thresholds for each price are calculated
    demand_thresholds = []
    for k in percentile_thresholds:
        demand_thresholds.append(np.percentile(test, k))

    # Every demand value is compared with the demand thresholds and assigned a demand-proportional price
    price_profile = []
    i = 0
    while i < len(test):
        j = 0
        while j < price_set_count:
            if test[i] <= demand_thresholds[j]:
                price_profile.append(price_set[j])
                break
            j = j + 1
        i = i + 1

    return price_profile

In [6]:
# Function for creating a price profile using the flat energy pricing method

def flat_energy_pricing(test, flat_price):
    total_hours = len(test)
    price_profile = []

    i = 0
    while i < total_hours:
        price_profile.append(flat_price)
        i = i + 1
    
    return price_profile

In [7]:
# Function for creating a price profile using the time of use energy pricing method

def tou_pricing(test, tou_dict):
    price_profile = []
    
    for Hour in test.index:
        [min_time, max_time] = tou_dict['time_range']
        [min_period_price, max_period_price] = tou_dict['prices']

        if (Hour.hour < max_time) & (Hour.hour >= min_time):
            price_profile.append(max_period_price)
        else:
            price_profile.append(min_period_price)
    
    return price_profile

In [8]:
# Function for measuring demand-proportionality of a pricing method using Pearson Correlation Coefficient

def demand_proportionality(demand, price_profile):
    # If any array has all identical values, the arrays do not correlate
    if len(set(demand)) == 1 or len(set(price_profile)) == 1:
        price_proportionality = float(0)
    else:
        # Proportionality measured using Pearson Correlation Coefficient
        price_proportionality = np.corrcoef(demand, price_profile)[0, 1]
    price_proportionality = np.round(price_proportionality, 3)
    return price_proportionality

In [9]:
# Function for running the demand proportionality simulation and collecting the results

def analyse(df, test_length, batch_count):
    # Find appropriate (random) start indices for batches to prevent overlap
    start_index_list = extract_batches(df, test_length, batch_count)
    
    # Create the empty results dataframe to hold the results
    df_results = pd.DataFrame(columns=['Energy (kWh)', 'Flat Energy', 'Time of Use', 'Range', 'Percentile'])

    for start_index in start_index_list:
        # Retrieve the batch test data
        test = df[start_index: start_index + test_length]

        # Calculate hourly demand profile in batch
        demand_value_profile = test['Energy']
        
        # Calculate total energy demand in batch
        demand_value = np.round(demand_value_profile.sum(), 2)

        # Calculate the price profile and demand proportionality value for each pricing method: 
        ## Flat Energy Pricing Method (0.23 dollars per kWh)
        flat_energy_price_profile = flat_energy_pricing(test['Energy'], 0.23)
        flat_energy_value = demand_proportionality(demand_value_profile, flat_energy_price_profile)

        ## Time of Use Pricing Method (0.25 dollars per kWh between 08:00 - 21:00, 0.2 dollars otherwise )
        tou_dict = {'time_range': [8, 21], 'prices': [0.20, 0.25]}
        tou_price_profile = tou_pricing(test['Energy'], tou_dict)
        tou_value = demand_proportionality(demand_value_profile, tou_price_profile)

        ## Range Pricing Method
        range_price_profile = range_pricing(test['Energy'], 20, 25)
        range_value = demand_proportionality(demand_value_profile, range_price_profile)

        ## Percentile Pricing Method
        percentile_price_profile = percentile_pricing(test['Energy'], 20, 25)
        percentile_value = demand_proportionality(demand_value_profile, percentile_price_profile)

        # Append the batch results to the results dataframe
        row = [demand_value, flat_energy_value, tou_value, range_value, percentile_value]
        df_results = pd.concat([df_results, pd.DataFrame([row], columns=df_results.columns)], ignore_index=True)

    # Calculate the average of each column
    average_value_list = np.round(df_results.mean(), 2).tolist()
    # Calculate the minimum of each column
    min_value_list = np.round(df_results.min(), 2).tolist()
    # Calculate the maximum of each column
    max_value_list = np.round(df_results.max(), 2).tolist()
    # Calculate the standard deviation of each column
    stddev_value_list = np.round(df_results.std(), 2).tolist()

    # Append the average to each column
    df_results = pd.concat([df_results, pd.DataFrame([average_value_list], columns=df_results.columns)], ignore_index=True)
    # Append the minimum to each column
    df_results = pd.concat([df_results, pd.DataFrame([min_value_list], columns=df_results.columns)], ignore_index=True)
    # Append the maximum to each column
    df_results = pd.concat([df_results, pd.DataFrame([max_value_list], columns=df_results.columns)], ignore_index=True)
    # Append the standard deviation to each column
    df_results = pd.concat([df_results, pd.DataFrame([stddev_value_list], columns=df_results.columns)], ignore_index=True)

    # Reformat the results dataframe
    df_results = df_results.transpose()
    df_results.columns = ['Batch 1', 'Batch 2', 'Batch 3', 'Batch 4', 'Batch 5', 'Batch 6', 'Batch 7', 'Batch 8', 'Batch 9', 'Batch 10',
                            'Average', 'Minimum', 'Maximum', 'Std. Dev.']
    
    # Display results
    display(df_results)

    return df_results

In [10]:
# Function for conducting extensive batch testing of demand proportionality for different periods, with 10 random batches each

def main():
    # Make the directory for the results
    if os.path.isdir('../../data/demand_proportionality_results'):
        pass
    else:
        os.mkdir('../../data/demand_proportionality_results')

    print('-- Demand proportionality simulation for different pricing methods --')
    for name in ['WEBSTER', 'CAMBRIDGE', 'MPL', 'RINCONADA LIB', 'TED THOMPSON', 'HAMILTON', 'HIGH', 'BRYANT']:

        df = read_data(name)
        print('\n{}'.format(name))
        file_name = '../../data/demand_proportionality_results/' + name + '.xlsx'

        # Write the tables to appropriate xlsx files
        with pd.ExcelWriter(file_name, mode='w', engine='xlsxwriter') as writer:
            
            # Modify execution parameters if needed
            test_length = [672, 336, 168, 72, 48, 24]                                     # Hours in batch
            batch_count = [10, 10, 10, 10, 10, 10]                                        # Batch count
            start_row = [0, 7, 14, 21, 28, 35]                                            # Row position in xlsx file to write results in
            durations = ['28 Days', '14 Days', '7 Days', '3 Days', '2 Days', '1 Day']     # Durations of batches, used for titles

            all_results_list = []

            i = 0
            while i < len(test_length):
                print('{} Revenue Simulation:'.format(durations[i]))

                # Run set of batches and collect results
                df_results = analyse(df, test_length[i], batch_count[i])
                all_results_list.append(df_results)

                # Write results to xlsx file
                df_results.to_excel(writer, sheet_name=name, startrow=start_row[i])
                pd.DataFrame(['{} Revenue'.format(durations[i])]).to_excel(writer, sheet_name=name, startrow=start_row[i], header=None, index=None)
                i = i + 1

            # Create summary dataframes
            df_summary_avg = pd.concat([df['Average'] for df in all_results_list], axis="columns")
            df_summary_min = pd.concat([df['Minimum'] for df in all_results_list], axis="columns")
            df_summary_max = pd.concat([df['Maximum'] for df in all_results_list], axis="columns")
            df_summary_stddev = pd.concat([df['Std. Dev.'] for df in all_results_list], axis="columns")

            for i in [df_summary_avg, df_summary_min, df_summary_max, df_summary_stddev]:
                i.columns = durations

            # Modify summary parameters if needed
            start_row = [42, 49, 56, 63]                                                        # Row position in xlsx file to write results in
            summary_name = [df_summary_avg, df_summary_min, df_summary_max, df_summary_stddev]  # Summary reference list
            print_title = ['Average', 'Minimum', 'Maximum', 'Standard Deviation']               # Title for printing results
            summary_title = ['Avg.', 'Min.', 'Max.', 'Std.Dev.']                                # Dataframe title in xlsx file

            i = 0
            while i < 4:
                print('{} Values Summary - {}:'.format(print_title[i], name))

                # Display summary
                display(summary_name[i])

                # Write summary to xlsx file
                summary_name[i].to_excel(writer, sheet_name=name, startrow=start_row[i])
                pd.DataFrame(['{} Summary'.format(summary_title[i])]).to_excel(writer, sheet_name=name, startrow=start_row[i], header=None, index=None)
                i = i + 1

            # Adjust the first column's width
            worksheet = writer.sheets[name]
            workbook = writer.book
            worksheet.set_column(0, 0, 15)
            # Adjust cell formating
            format1 = workbook.add_format({'bg_color': '#EDEDED', 'num_format': '#,##0.00', 'border': 1})
            worksheet.conditional_format(0, 0, 68, 14, {'type': 'no_blanks', 'format': format1})
main()

-- Demand proportionality simulation for different pricing methods --



WEBSTER
28 Days Revenue Simulation:


,Batch 1,Batch 2,Batch 3,Batch 4,Batch 5,Batch 6,Batch 7,Batch 8,Batch 9,Batch 10,Average,Minimum,Maximum,Std. Dev.
Energy (kWh),6461.360,4470.500,5145.900,4512.070,7848.190,4678.380,7806.770,4569.640,1338.020,2650.730,4948.16,1338.02,7848.19,2050.92
Flat Energy,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.00,0.00,0.00
Time of Use,0.647,0.638,0.678,0.648,0.698,0.679,0.656,0.703,0.450,0.495,0.63,0.45,0.70,0.09
Range,0.982,0.975,0.981,0.980,0.982,0.979,0.978,0.984,0.954,0.982,0.98,0.95,0.98,0.01
Percentile,0.967,0.908,0.943,0.917,0.982,0.926,0.982,0.918,0.795,0.922,0.93,0.80,0.98,0.05


14 Days Revenue Simulation:


,Batch 1,Batch 2,Batch 3,Batch 4,Batch 5,Batch 6,Batch 7,Batch 8,Batch 9,Batch 10,Average,Minimum,Maximum,Std. Dev.
Energy (kWh),3136.170,1860.310,2711.740,2355.100,3878.360,1997.780,3893.650,2421.250,715.710,2434.130,2540.42,715.71,3893.65,951.72
Flat Energy,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.00,0.00,0.00
Time of Use,0.628,0.600,0.722,0.624,0.722,0.602,0.631,0.737,0.437,0.679,0.64,0.44,0.74,0.09
Range,0.983,0.981,0.984,0.983,0.987,0.974,0.979,0.985,0.956,0.983,0.98,0.96,0.99,0.01
Percentile,0.963,0.891,0.950,0.923,0.985,0.913,0.980,0.924,0.777,0.938,0.92,0.78,0.98,0.06


...

,28 Days,14 Days,7 Days,3 Days,2 Days,1 Day
Energy (kWh),1372.00,756.53,366.16,162.50,112.88,60.04
Flat Energy,0.00,0.00,0.00,0.00,0.00,0.00
Time of Use,0.50,0.49,0.47,0.40,0.41,0.36
Range,0.94,0.95,0.95,0.97,0.97,0.98
Percentile,0.87,0.80,0.84,0.77,0.77,0.86


Maximum Values Summary - BRYANT:


,28 Days,14 Days,7 Days,3 Days,2 Days,1 Day
Energy (kWh),6014.59,3204.29,1723.19,724.81,527.03,241.68
Flat Energy,0.00,0.00,0.00,0.00,0.00,0.00
Time of Use,0.66,0.68,0.69,0.76,0.78,0.80
Range,0.98,0.98,0.99,0.99,0.99,0.99
Percentile,0.98,0.98,0.98,0.98,0.98,0.97


Standard Deviation Values Summary - BRYANT:


,28 Days,14 Days,7 Days,3 Days,2 Days,1 Day
Energy (kWh),1582.24,783.02,449.87,202.51,133.98,64.73
Flat Energy,0.00,0.00,0.00,0.00,0.00,0.00
Time of Use,0.06,0.07,0.08,0.13,0.15,0.15
Range,0.01,0.01,0.01,0.00,0.00,0.00
Percentile,0.04,0.05,0.04,0.06,0.07,0.04
